In [1]:
from urllib.request import urlopen
import json
import pandas as pd
from datetime import datetime
from random import uniform
from time import sleep

Pour ce faire nous commençons par importer une basée de données reprennant toutes les communes de France:
[Base de données villes française CC-BY-SA](https://sql.sh/736-base-donnees-villes-francaises)


In [2]:
communes = pd.read_csv("data/villes_france.csv", sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
communes.head(3)

,ville_id,ville_departement,ville_slug,ville_nom,ville_nom_simple,ville_nom_reel,ville_nom_soundex,ville_nom_metaphone,ville_code_postal,ville_commune,...,ville_densite_2010,ville_surface,ville_longitude_deg,ville_latitude_deg,ville_longitude_grd,ville_latitude_grd,ville_longitude_dms,ville_latitude_dms,ville_zmin,ville_zmax
0,1,01,ozan,OZAN,ozan,Ozan,O250,OSN,01190,284,...,93,6.6,4.91667,46.3833,2866,51546,+45456,462330,170,205
1,2,01,cormoranche-sur-saone,CORMORANCHE-SUR-SAONE,cormoranche sur saone,Cormoranche-sur-Saône,C65652625,KRMRNXSRSN,01290,123,...,107,9.85,4.83333,46.2333,2772,51379,+44953,461427,168,211
2,3,01,plagne-01,PLAGNE,plagne,Plagne,P425,PLKN,01130,298,...,20,6.2,5.73333,46.1833,3769,51324,+54342,461131,560,922


Sur les 27 colomnes, nous garderons que les suivantes:

In [3]:
colmnuns = [
    "ville_id",
    "ville_departement",
    "ville_nom",             # En majuscule sans accents
    "ville_nom_simple",      # En minuscule sans accents ni tirets
    "ville_nom_reel", 
    "ville_code_postal",
    "ville_commune",
    "ville_code_commune",    # ou INSEE
    "ville_arrondissement",
    "ville_canton",
    "ville_amdi",
    "ville_population_2010",
    "ville_population_1999",
    "ville_population_2012",
    "ville_densite_2010",
    "ville_surface",
    "ville_longitude_deg",
    "ville_latitude_deg",
    "ville_zmax"              # L'altitude
]           

communes = communes.filter(colmnuns)
communes.head(3)

,ville_id,ville_departement,ville_nom,ville_nom_simple,ville_nom_reel,ville_code_postal,ville_commune,ville_code_commune,ville_arrondissement,ville_canton,ville_amdi,ville_population_2010,ville_population_1999,ville_population_2012,ville_densite_2010,ville_surface,ville_longitude_deg,ville_latitude_deg,ville_zmax
0,1,01,OZAN,ozan,Ozan,01190,284,01284,2,26,6,618,469,500,93,6.6,4.91667,46.3833,205
1,2,01,CORMORANCHE-SUR-SAONE,cormoranche sur saone,Cormoranche-sur-Saône,01290,123,01123,2,27,6,1058,903,1000,107,9.85,4.83333,46.2333,211
2,3,01,PLAGNE,plagne,Plagne,01130,298,01298,4,03,6,129,83,100,20,6.2,5.73333,46.1833,922


Maintenant nous allons reprendre uniquement les communes des départements suivants:

In [15]:
departements = [
    "31",  # Haute-Garonne
    "32",  # Gers
    "65",  # Hautes-Pyrénées
    "81",  # Tarn
]

toulouse_counrtyside = communes.loc[communes['ville_departement'].isin(departements)]
toulouse_counrtyside['ville_latitude_deg'] = toulouse_counrtyside['ville_latitude_deg'].astype('float32')
toulouse_counrtyside['ville_longitude_deg'] = toulouse_counrtyside['ville_longitude_deg'].astype('float32')

toulouse_counrtyside.head(3)

,ville_id,ville_departement,ville_nom,ville_nom_simple,ville_nom_reel,ville_code_postal,ville_commune,ville_code_commune,ville_arrondissement,ville_canton,ville_amdi,ville_population_2010,ville_population_1999,ville_population_2012,ville_densite_2010,ville_surface,ville_longitude_deg,ville_latitude_deg,ville_zmax
11514,11515,31,LUNAX,lunax,Lunax,31350,307,31307,2,06,6,60,56,100,11,5.1,0.683333,43.333302,298
11515,11516,31,FONTENILLES,fontenilles,Fontenilles,31470,188,31188,1,30,6,4505,2920,3900,222,20.22,1.200000,43.549999,280
11516,11517,31,ARBON,arbon,Arbon,31160,012,31012,2,01,6,92,76,100,20,4.47,0.750000,43.000000,896


In [5]:
toulouse_counrtyside.count()

ville_id                 1849
ville_departement        1849
ville_nom                1849
ville_nom_simple         1849
ville_nom_reel           1849
ville_code_postal        1849
ville_commune            1849
ville_code_commune       1849
ville_arrondissement     1849
ville_canton             1849
ville_amdi               1849
ville_population_2010    1849
ville_population_1999    1849
ville_population_2012    1849
ville_densite_2010       1849
ville_surface            1849
ville_longitude_deg      1849
ville_latitude_deg       1849
ville_zmax               1849
dtype: int64

In [24]:
gares = pd.read_csv("data/localisation-des-gares-et-haltes-ferroviaires-doccitanie.csv", sep=';', error_bad_lines=False, index_col=False, dtype='unicode')
departements = [
    "31",  # Haute-Garonne
    "32",  # Gers
    "65",  # Hautes-Pyrénées
    "81",  # Tarn
        ]

codeuic = [
"87615476",
"87615484",
"87615468",
"87615526",
"87775197",
"87328021",
"87615542",
"87615013",
"87618215",
"87611723",
"87618207",
"87611707",
"87612028",
"87612002",
"87616011",
"87611400",
"87611301",
"87611665",
"87611657",
"87446179",
"87353573",
"87611467",
"87611962",
"87611004",
"87618116",
"87611384",
"87353581",
"87611434",
"87611954",
"87611392",
"87353599",
"87611947",
"87497461",
"87612010",
"87611939",
"87611376",
"87611921"
]

gares = gares.loc[gares['departement'].isin(departements)]
#gares = gares.loc[~gares['codeuic'].isin(codeuic)]
gares.head(3)

,objectid,id_rte500,nature,toponyme,type,codeuic,region,departement,date_maj,geo_shape,geo_point_2d
3,16,1595,Gare de voyageurs et de fret,Colomiers,gare avec accueil du public,87611467,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.3340385138...","43.6032693934,1.3340385139"
11,46,2064,Gare de voyageurs,L'Isle-Jourdain,gare avec accueil du public,87611806,Occitanie,32,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.0897045076...","43.6173114196,1.08970450764"
13,51,2088,Gare de voyageurs et de fret,Cazères-sur-Garonne,gare avec accueil du public,87611079,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.0823282310...","43.2127599987,1.08232823102"


In [28]:
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in gares['geo_point_2d']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(float(row.split(',')[0]))
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(float(row.split(',')[1]))
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
gares['latitude'] = lat
gares['longitude'] = lon

Let's see the distance between two points

In [25]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r
haversine(0.683333,43.333302,1.3340385139, 43.6032693934)

60.48662575516235

In [29]:
df_all = pd.merge(toulouse_counrtyside.assign(key=0), gares.assign(key=0), on='key').drop('key', axis=1)

In [42]:
df_all['DISTANCE_GARE'] = df_all.apply(
    (lambda row: haversine(
        row['ville_longitude_deg'], row['ville_latitude_deg'],
        row['longitude'], row['latitude']
    )),
    axis=1
)

In [41]:
toulouse_counrtyside['ville_longitude_deg'].count()*gares['latitude'].count()
df_all['DISTANCE_GARE'].count()
print(toulouse_counrtyside['ville_longitude_deg'].iloc[3], toulouse_counrtyside['ville_latitude_deg'].iloc[3])
print(gares['latitude'].iloc[3], gares['longitude'].iloc[3])


0.566667 43.15
43.6561845416 1.97438119387


In [43]:
closest = df_all.loc[df_all.groupby(["ville_id", 'ville_latitude_deg', 'ville_longitude_deg'])["DISTANCE_GARE"].idxmin()]

In [44]:
toulouse_counrtyside_w_dist = toulouse_counrtyside.merge(
    closest,
    on=["ville_id", 'ville_latitude_deg', 'ville_longitude_deg'],
    suffixes=('', '_cl')
).drop(['latitude', 'latitude'], axis=1)

In [45]:
toulouse_counrtyside_w_dist.sort_values(by=['DISTANCE_GARE']).head(7)

,ville_id,ville_departement,ville_nom,ville_nom_simple,ville_nom_reel,ville_code_postal,ville_commune,ville_code_commune,ville_arrondissement,ville_canton,...,toponyme,type,codeuic,region,departement,date_maj,geo_shape,geo_point_2d,longitude,DISTANCE_GARE
103,11618,31,FENOUILLET,fenouillet,Fenouillet,31150,182,31182,3,49,...,Fenouillet-Saint-Alban,halte ferroviaire,87611962,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.4013341146...","43.6843233667,1.40133411461",1.401334,0.156389
555,12070,31,MONTASTRUC-LA-CONSEILLERE,montastruc la conseillere,Montastruc-la-Conseillère,31380,358,31358,3,19,...,Montastruc-la-Conseillère,halte ferroviaire,87615369,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.5974276883...","43.7157469018,1.59742768831",1.597428,0.232393
313,11828,31,AVIGNONET-LAURAGAIS,avignonet lauragais,Avignonet-Lauragais,31290,037,31037,3,38,...,Avignonet,halte ferroviaire,87616011,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.7876235476...","43.3639242339,1.78762354766",1.787624,0.242190
1809,33330,81,CARMAUX,carmaux,Carmaux,81400,060,81060,1,97,...,Carmaux,gare avec accueil du public,87615625,Occitanie,81,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [2.1523689320...","44.048256148,2.15236893204",2.152369,0.270941
336,11851,31,LABEGE,labege,Labège,31670,254,31254,3,10,...,Labège-Village,halte ferroviaire,87612028,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.5333464811...","43.5300993993,1.53334648112",1.533346,0.355725
1687,33208,81,VIELMUR-SUR-AGOUT,vielmur sur agout,Vielmur-sur-Agout,81570,315,81315,2,35,...,Vielmur-sur-Agout,halte ferroviaire,87775197,Occitanie,81,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [2.0894428718...","43.6240880687,2.08944287185",2.089443,0.387672
30,11545,31,CAZERES,cazeres,Cazères,31220,135,31135,1,11,...,Cazères-sur-Garonne,gare avec accueil du public,87611079,Occitanie,31,2018-02-07T02:00:00+01:00,"{""type"": ""Point"", ""coordinates"": [1.0823282310...","43.2127599987,1.08232823102",1.082328,0.445731


In [14]:
toulouse_counrtyside_w_dist.to_csv('data/tls-communes-gares.csv', encoding='utf-8')